### 0. 事前準備

特にありません。

In [ ]:
%matplotlib inline

import branca.colormap as cm
from decimal import Decimal, ROUND_HALF_UP
import folium
from geopy.distance import geodesic
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import re
import requests
import seaborn as sns

### 1. データ型の変換

Wikipediaの「List of people who died climbing Mount Everest」からエベレスト登山死者データを取得します。

URL:  
https://en.wikipedia.org/wiki/List_of_people_who_died_climbing_Mount_Everest

データの取得方法:  
`pd.read_html()`に上記URLを指定するとページ上にあるすべての`table`タグのデータがデータフレームの配列として取得できますので、そこから必要なデータフレームを選択します。

ただし、Pythonのバージョンによってはエラーとなってデータが取得できない場合があります。
その場合は以下のGitHubのスレッドにあるように、`requests.get()`でHTMLを取得してから`pd.read_html()`を試してください。  
https://github.com/pandas-dev/pandas/issues/21499

データの保存:  
取得後はセルを実行のたびに何度も取得しないよう、該当データフレームを`to_csv()`でCSVファイルにして、`data/list_of_people_who_died_climbing_mount_everest.csv`に保存します。

次回以降のセル実行ではファイルが存在する場合、ファイルからデータを読み込むようにします。

In [ ]:
everest_death_wikipedia_url = 'https://en.wikipedia.org/wiki/List_of_people_who_died_climbing_Mount_Everest'
everest_death_csv_path = 'data/list_of_people_who_died_climbing_mount_everest.csv'

try:
    everest_death_df = pd.read_csv(everest_death_csv_path)
    print(f"ファイル'{everest_death_csv_path}'からデータを取得しました。")
except FileNotFoundError:
    print(f"ファイル'{everest_death_csv_path}'が存在しないためウィキペディアからデータを取得します。")
    everest_death_html = requests.get(everest_death_wikipedia_url)
    everest_death_df = pd.read_html(everest_death_html.text)[1]
    everest_death_df.to_csv(everest_death_csv_path, index=False)
    
len(everest_death_df)

データの中身を確認。

In [ ]:
everest_death_df.head(10)

特に問題なくデータを取得できているようですが、`Refs`列は必要ないので削除します。

In [ ]:
everest_death_df.drop(columns=['Refs'], inplace=True)

everest_death_df

データの型と欠損を確認。

In [ ]:
everest_death_df.info()

#### 1-1. 日時型への変換

まずは日付の変換をします。

`Date`は日時型ではないようですので日付のフォーマットを指定して日時型に変換し、新たな列の`death_date`に格納します。

参考）  
strftime() と strptime() の振る舞い  
https://docs.python.org/ja/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
everest_death_df['death_date'] = pd.to_datetime(everest_death_df['Date'], format="%B %d, %Y", errors='coerce')

everest_death_df['death_date']

変換失敗がないか確認。

In [ ]:
print("「Date」のNA数:", everest_death_df['Date'].isna().sum())
print("「death_date」のNA数:", everest_death_df['death_date'].isna().sum())

失敗が２件あるようなので、どんなデータで失敗しているかを確認。

In [ ]:
everest_death_df[everest_death_df['death_date'].isna()]

これはデータの問題なので、`death_date`を直接設定してしまいます。

In [ ]:
na_index_list = everest_death_df[everest_death_df['death_date'].isna()].index

everest_death_df.loc[na_index_list[0], 'death_date'] = pd.Timestamp(year=1934, month=5, day=31)

everest_death_df.loc[na_index_list[1], 'death_date'] = \
    pd.Timestamp(year=1975, month=8, day=22) # 死亡日は22あたりにしておきます。

everest_death_df.iloc[na_index_list]

もう一度変換失敗がないか確認。

In [ ]:
everest_death_df['Date'].isna().sum()

死亡年をX軸、死亡者数をY軸とした折れ線グラフを描いてみましょう。

死亡年でグループ分けし、インデックスが`death_year`、列が`死亡者数`のデータフレームを作成します。

In [ ]:
everest_death_groupby_death_year = everest_death_df.groupby(everest_death_df['death_date'].dt.year) \
                                    .agg(死亡者数=('No.', 'count'))

# インデックス名を変更。
everest_death_groupby_death_year.index.names = ['death_year']

everest_death_groupby_death_year.head(10)

`death_year`の値が飛び飛びなので、欠けている年をすべて埋めることとします。

まずはインデックスを日付型（各年の元旦）に変換します。

In [ ]:
everest_death_groupby_death_year.index = pd.to_datetime(everest_death_groupby_death_year.index, format="%Y")

everest_death_groupby_death_year.head()

インデックスを日付型に変換したら、今度は`pd.asfreq()`を使って年の穴埋めを行います。

`pd.asfreq()`の引数には以下を設定します。  
・`freq`: 頻度を表すコード。ここでは毎年の開始日を表す`YS`を指定。  
・`fill_value`: 値の穴埋め。死亡者無しなので0とします。

In [ ]:
everest_death_groupby_death_year = everest_death_groupby_death_year.asfreq(freq='YS', fill_value=0)

In [ ]:
everest_death_groupby_death_year

それではグラフを描画します。

どんなことが読み取れますか？

In [ ]:
sns.set(font=['MS Gothic', 'Hiragino Sans', 'TakaoPGothic']) # 日本語フォントは設定が必要

fig, ax = plt.subplots(figsize=(18, 10))

sns.lineplot(data=everest_death_groupby_death_year, x="death_year", y="死亡者数")

#### 1-2. 文字列型からカテゴリ型への変換

今度は集計やグラフで使いやすいよう、`Nationality`をカテゴリ型に変換します。

カテゴリ数が少ない場合、カテゴリ型に変換することでデータのサイズを小さくすることができます。

まずはどんなデータが入っているか確認。

何か読み取れることがありますか？

In [ ]:
everest_death_df['Nationality'].value_counts()

カテゴリにするには少々数が多すぎるかもしれません。  
一方で、`Nepal`、`India`の数が極端に多いことが分かります。

そこで、今回は`Nepal`、`India`、`Others`の３つのカテゴリに分けることとします。

まずはすべてのデータをカテゴリ型に変換します。

In [ ]:
everest_death_df['nationality_cat'] = everest_death_df['Nationality'].astype('category')

everest_death_df['nationality_cat']

新たに`Others`というカテゴリを追加します。

In [ ]:
everest_death_df['nationality_cat'] = everest_death_df['nationality_cat'].cat.add_categories('Others')

everest_death_df['nationality_cat'].cat.categories

次に`Nepal`, `India`以外のデータをすべて`Others`に置き換えてしまいます。

その後、実際に変更されたことを`value_count()`で確認しましょう。

In [ ]:
everest_death_df.loc[~everest_death_df['nationality_cat'].isin(['Nepal', 'India']), 'nationality_cat'] = 'Others'

everest_death_df['nationality_cat'].value_counts()

`value_counts()`で件数が0でも表示されるのは、カテゴリが存在するためです。

未使用のカテゴリを`remove_unused_categories()`で削除し、もう一度`value_count()`を実行して削除されていることを確認します。

In [ ]:
everest_death_df['nationality_cat'] = everest_death_df['nationality_cat'].cat.remove_unused_categories()

everest_death_df['nationality_cat'].value_counts()

先ほどと同じように、死亡年をX軸、死亡者数をY軸とした折れ線グラフを描いてみましょう。  
ただし、今回は`nationality_cat`ごとに線を描画します。

死亡年と`nationality_cat`でグループ化し、インデックスが`death_year`と`nationality_cat`、列が`死亡者数`のデータフレームを作成します。

In [ ]:
groupby_death_year_nationality = everest_death_df \
                                    .groupby([everest_death_df['death_date'].dt.year, 'nationality_cat']) \
                                    .agg(死亡者数=('No.', 'count'))

# インデックス名を変更。
groupby_death_year_nationality.index.names = ['death_year', 'nationality_cat']

groupby_death_year_nationality.head(10)

先ほどと同じようにインデックスの死亡年の欠けている年をすべて埋め、死亡者数を0とします。

In [ ]:
# death_yearの日付型への変換がしやすいよう、nationality_catをインデックスから外しておく。
groupby_death_year_nationality = groupby_death_year_nationality.unstack(1)

groupby_death_year_nationality.index = pd.to_datetime(groupby_death_year_nationality.index, format="%Y")

groupby_death_year_nationality = groupby_death_year_nationality.asfreq('YS', fill_value=0)

groupby_death_year_nationality.head()

最後に`death_year`、`nationality_cat`をインデックスから外し、`death_year`、`nationality_cat`、`死亡者数`の３つの列となるようにします。

In [ ]:
groupby_death_year_nationality = groupby_death_year_nationality.stack().reset_index()

groupby_death_year_nationality

それでは死亡年をX軸、死亡者数をY軸とした折れ線グラフを描画します。  
凡例の順番を揃えるよう、`lineplot()`の引数に`hue_order=['Nepal', 'India', 'Others']`を渡します。

どんなことが読み取れますか？

In [ ]:
### チャレンジしてみよう！！ ###

### 2. 位置座標の変換

ここからはウィキペディアの「世界の山一覧 (高さ順)」にある山のデータを取得して、地図やグラフに表示してみましょう。

URL:  
[https://ja.wikipedia.org/wiki/世界の山一覧_(高さ順)](https://ja.wikipedia.org/wiki/世界の山一覧_(高さ順\))  

データの取得方法:  
`pd.read_html()`に上記URLを指定するとページ上にあるすべての`table`タグのデータがデータフレームの配列として取得できますので、そこから必要なデータフレームを選択します。

ただし、Pythonのバージョンによってはエラーとなってデータが取得できない場合があります。
その場合は以下のGitHubのスレッドにあるように、`requests.get()`でHTMLを取得してから`pd.read_html()`を試してください。  
https://github.com/pandas-dev/pandas/issues/21499


データの保存:  
取得後はセルを実行のたびに何度も取得しないよう、該当データフレームを`to_csv()`でCSVファイルにして、`data/highest_montains.csv`に保存します。  
次回以降のセル実行ではファイルが存在する場合、ファイルからデータを読み込むようにします。

In [ ]:
highest_montains_wikipedia_url = 'https://ja.wikipedia.org/wiki/世界の山一覧_(高さ順)'
highest_montains_csv_path = 'data/highest_montains.csv'

try:
    highest_montain_df = pd.read_csv(highest_montains_csv_path)
    print(f"ファイル'{highest_montains_csv_path}'からデータを取得しました。")
except FileNotFoundError:
    print(f"ファイル'{highest_montains_csv_path}'が存在しないためウィキペディアからデータを取得します。")
    highest_montains_html = requests.get(highest_montains_wikipedia_url)
    highest_montain_df = pd.read_html(highest_montains_html.text)[1]
    highest_montain_df.to_csv(highest_montains_csv_path, index=False)
    
len(highest_montain_df)

データの中身を確認。

In [ ]:
highest_montain_df.head(10)

さっと見て、明らかにおかしい以下の二点を修正します。

1.エベレストの位置座標  
　先頭に余計なコードが含まれてしまっているようなので除去します。
 
2.存在しない列「Unnamed: 9」  
　削除します。

In [ ]:
highest_montain_df.loc[0, '位置'] = re.sub(r'^.+?(?=北緯)', '', highest_montain_df.loc[0, '位置'])

highest_montain_df.drop(columns=['Unnamed: 9'], inplace=True)

highest_montain_df.head()

データの型と欠損を確認します。

順位に欠損が少々あります。

In [ ]:
highest_montain_df.info()

山を地図上で表示できるよう、位置座標の変換を行います。

まずはデータを確認。

In [ ]:
highest_montain_df['位置']

フォーマットと「度・分・秒」と「度のみ」の両方の表記があることが分かりました。  
（ブラウザでは「度・分・秒」表記しか見えないのでやや肩透かしでありますが、「度のみ」の数値を抽出します。）

数値抽出のための正規表現を作り、列の中でマッチしないデータがないか確認します。

In [ ]:
regex = r'.+\\ufeff / \\ufeff北緯(\d+\.\d)度 東経(\d+\.\d)度'

regex = r'^.+北緯(\d{2}\.\d{5})度 東経(\d{2,3}\.\d{5})度$'
(~highest_montain_df['位置'].str.match(regex)).sum()

すべてにマッチするようであれば、`Series.str.extract()`で数値抽出をおこないます。

抽出した数値はそれぞれ、新たな列`latitude`、`longitude`に格納します。

In [ ]:
highest_montain_df[['latitude', 'longitude']] = highest_montain_df['位置'].str.extract(regex).astype(np.float64)

highest_montain_df[['位置', 'latitude', 'longitude']]

### 3. 地図上にデータを表示

山を以下のルールに沿って地図上に表示してみます。
- 山の位置に円を配置。
- 円の色を高さに合わせて滑らかに色分けする。

まず滑らかな色分けのためのカラーマップを作成します。

`cm.LinearColormap`クラスのコンストラクタに任意の色の配列、山の高さの最小値、最大値を渡してインスタンスを生成します。

In [ ]:
min_elevation = highest_montain_df['高さ (メートル)'].min()
max_elevation = highest_montain_df['高さ (メートル)'].max()

linear = cm.LinearColormap(['yellow', 'orange', 'red'], vmin=min_elevation, vmax=max_elevation)

それでは地図上に表示します。

どんなことが読み取れますか？

In [ ]:
everest_coords = tuple(highest_montain_df.loc[0, ['latitude', 'longitude']].to_list())

map = folium.Map(location=everest_coords, zoom_start=4, tiles="cartodbdark_matter") # 地味な地図を選択。

# 高い山が上に重なるようにソート。
for i, row in highest_montain_df.sort_values('高さ (メートル)', ascending=True).iterrows():

    popup_message_html = f"""
    <p>山名: {row['山名']}</p>
    <p>順位: {'' if math.isnan(row['順位']) else int(row['順位'])}</p>
    <p>高さ: {row['高さ (メートル)']:,}m</p>
    <p>位置: ({row['latitude']}, {row['longitude']})</p>
    <p>初登頂年号: {row['初登頂年号']}</p>
    """
    popup = folium.Popup(folium.IFrame(popup_message_html), min_width=400, max_width=400)

    color = linear(row['高さ (メートル)'])
    
    folium.Circle(location=(row['latitude'], row['longitude']),
                  radius=2000,
                  color=color,
                  fill_color=color,
                  fill_opacity=1,
                  weight=1.5,
                  popup=popup
                 ).add_to(map)

map

地図では山の高さが認識しにくいので、棒グラフでも表示してみましょう。  
地図と見比べやすいようにX軸を経度、Yを高さとします。

8,000m峰はいくつありますか？  
また、他にどんなことが読み取れますか？

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))

sns.barplot(x='山名', y='高さ (メートル)', data=highest_montain_df.sort_values('longitude', ascending=True))
plt.xticks(rotation=90);